# 模型介绍

**End-to-End Object Detection with Transformers**



参考代码: https://aistudio.baidu.com/aistudio/projectdetail/1327221

* 在大佬的基础上进行修改，包括Paddle 2.0适配， BUG修复，以及精度对齐等工作



github pytorch代码: [https://github.com/facebookresearch/detr](https://github.com/facebookresearch/detr)

论文地址: [https://arxiv.org/pdf/1706.03762.pdf](https://arxiv.org/pdf/1706.03762.pdf)
- 基于transformer实现端到端目标检测, 两大主要组成，一者是预测的匹配损失函数，再者预测目标和他们的关系。
- 不需要使用NMS或者anchor生成， 本文能将该任务的先验知识进行编码。DETR结合目标关联关系和上下文直接并行输出最终的预测集。
- Bipartite Matching二分图最大匹配， 将节点集V分割为互不相割的子集。训练时将预测框和GT boxes进行匹配。若没有找到匹配的预测框作为"no object"。 最终计算匹配损失。
- transformer的自注意力机制，专门对序列中成对交互的元素进行建模，所以也很好解决去除重复框
- 推理出的box坐标是直接基于输入图片而不是基于anchor
- 先预设固定数量的预测量，通常比图像的目标数量多。南调是对预测目标进行评分，使用最低成对匹配成本，用到匈牙利算法。线性组合IOU损失和L1损失。
- 网络结构由cnn(res50)+transformer+FFN. 常用的backbone都可以用。cnn到transformer通过1x1卷积进行降维，空间维度压缩到一维， FFN由3层MLP+relu组成。
- backbone和transformer分开训练，学习率设置不一样。具体是怎样的方式？
- 关于transformer， 
	- 其实就是全连接(或一维卷积)加上attention结合体，对于局部特征捕获能力稍欠缺。
	- 编解码结构较大的局限性是固定长度的语义向量， 编码器压缩成一个固长向量无法表示整个序列信息, 前期信息容易被后序信息覆盖，解码时输入每个单词权重不一致。
	- attention解决编解码结构的弊端，编码器时将正序和逆序的隐藏层状态结合起来， 解码时步骤，1.计算编码器隐藏层状态和解码器隐藏层状态间的相关程度并用softmax归一化得到权重值， 2. 计算语义编码向量的加权和，3. 根据加权和计算隐藏状态和解码器输出。不同编解码结构的是解码时选择性的从向量序列中挑选。	
	- soft attention缺点是每次decode都会计算所有编码器隐藏层状态向量，计算复杂度较高。hard attention每次近选择一个source进行计算，缺点是不可微，无法反向传播。
	- global attetion也属于soft attention， 只是计算权重公式上有差别。实验表明general方式好一些。
	- local attention, 每次选择一部分source进行计算， 既减少计算量又能可微。思路是为decoder预测一个source位置，然后基于位置选择一个窗口用于计算编码向量。关键是怎么确定pt位置， monotoic或predictive。
	- self attention, 传统attention是基于target和source间的相似度，而self是发生在target内部或者source内部的相似关联。self更容易捕获长距依赖特征。query和key点积相当于给每个单词打分，决定了编码单词时重视句子其他部分的程度。softmax决定了其他单词对当前单词编码的贡献，self作为最大，其他单词也很有参考作用。
	- transformer和attention机制一样，只是更复杂，多个encoder和docoder堆叠一起， encoder包含self-attention(不仅仅当前词还有上下文)和神经网络层(可并行)，decoder多了一层attention(当前需关注的重点内容)。encoder前要对输入数据进行embedding操作。
	- 位置编码， transformer缺少解释序列中单词顺序的方法，positional encoding在维度上和embedding一样，偶数位置正弦编码，奇数位置余弦编码，决定当前词的位置或不同词间距离，并且模型可学习到。
	- 多头注意力，主要扩展了模型专注不同位置的能力，给出了注意力层的多个表示子空间。8个头就有8套qkv矩阵，每个头的qkv矩阵权重独立，将这些矩阵拼接在一起并用附加权重矩阵相乘。
	- transformer还使用到了残差和layer norm。LN是在每一个样本上计算均值和方差，而不是BN那种在批方向计算均值和方差。还用到了mask，掩盖值，使其参数更新时不产生效果， padding mask在较短序列填充0，Sequence mask只依赖t之前的信息，将t之后信息掩盖起来。

# 关于数据集COCO2017

COCO的全称是Common Objects in Context，是微软团队提供的一个可以用来进行图像识别的数据集。MS COCO数据集中的图像分为训练、验证和测试集。其行业地位就不再多少了，本文主要梳理一下该数据集包含的内容。下图是官网给出的可下载的数据集（更新时间2020年01月09日），从这里可看出其数据集主要包括有标注的和无标注的数据。



![](https://ai-studio-static-online.cdn.bcebos.com/fc21793a8cd6402a9c9d8b9e630fc06d6696c4fc98504c2b982840445aef44a3)


In [ ]:
#解压数据集
%cd /home/aistudio/data/data7122/ 
!unzip train2017.zip
!unzip val2017.zip 
!unzip annotations_trainval2017.zip 

In [ ]:
#加载数据集
%cd ~/my_detr
!python coco_dataset.py

# 模型结构搭建

1. Backbone在resnet50基础上修改，支持DC5变体
2. 搭建transformer结构
3. 搭建后处理包括匈牙利匹配算法
4. 损失函数
4. 后处理



**核心代码主要有:**
* model.py
* resnet.py
* backbone.py
* transformer.py

![](https://ai-studio-static-online.cdn.bcebos.com/1161d69bda9049b2aa9659d62201b037de3d59c2afd94cb2af9bbe67201256a6)


# 精度对齐

因为loss函数以及后处理较为复杂，所以这里特定还用了loss精度对齐

## 输出结果结果精度对齐

模拟tensor输入:
image = [paddle.ones(shape=[3, 800, 1199])]
samples.tensors = torch.ones(1, 3, 800, 1199)
因为resize对图像输入有一定差距，所以使用ones的张量, 比如使用样例数据, 0.348(torch) vs 0.31(paddle)


pytorch结果:

![](https://ai-studio-static-online.cdn.bcebos.com/3001d9f670224454bf7762d037b31a7b77e5e3d8f5f247929a17a11d98fcfbc1)

paddle结果:

![](https://ai-studio-static-online.cdn.bcebos.com/24ad4a093a1e488cbb3fecef40182dc0f303cbe3a7f24b858f19c3166ed75417)




## LOSS 精度对齐

相差小数点2位

**Pytorch的结果:**

![](https://ai-studio-static-online.cdn.bcebos.com/dd6f3f4a7efa4b73a1eaf035fdc314b54de1e89e63854827bd12d2f11a462b0f)

**PaddlePaddle的结果:**

![](https://ai-studio-static-online.cdn.bcebos.com/5aad26eed0ec4f539aec384989aa3dd51394b4aca3f24df58f0d532497712018)



# 训练DETR

In [ ]:
#开始训练
%cd ~/my_detr
!python train_val.py train

/home/aistudio/my_detr
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/pytho

# 验证DETR

In [14]:
%cd ~/my_detr
!python train_val.py eval

/home/aistudio/my_detr
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/pytho

# 训练DETR-DC5

这个模型非常耗显存, 很难训练

In [ ]:
#开始训练DC5
%cd ~/my_detr
!python train_val_dc5.py train

/home/aistudio/my_detr
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/pytho

# 验证DETR-DC5

In [15]:
#验证DETR-DC5
%cd ~/my_detr
!python train_val_dc5.py eval

/home/aistudio/my_detr
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/pytho